In [1]:
import pickle
from sklearn.metrics import pairwise
import numpy as np
from tqdm import tqdm

## DATA

In [2]:
with open('/Users/dgknrsln/Documents/pythonProject/EXPERIMENTS/LATEST_test_docs.pkl', 'rb') as f:
    docs = pickle.load(f)

with open('/Users/dgknrsln/Documents/pythonProject/EXPERIMENTS/LATEST_test_queries.pkl', 'rb') as f:
    queries = pickle.load(f)

with open('/Users/dgknrsln/Documents/pythonProject/EXPERIMENTS/LATEST_test_data.pkl', 'rb') as f:
    pair = pickle.load(f)

## SGPT

In [3]:
with open('/Users/dgknrsln/Documents/pythonProject/EXPERIMENTS/LATEST/LATEST_sgpt_inference_docs.pkl', 'rb') as f:
    parent_embeddings_list = pickle.load(f)

with open('/Users/dgknrsln/Documents/pythonProject/EXPERIMENTS/LATEST/LATEST_sgpt_inference_queries.pkl', 'rb') as f:
    child_embeddings_list = pickle.load(f)

In [4]:
cos_matrix = pairwise.cosine_similarity(child_embeddings_list, parent_embeddings_list)

### 1.MAP

In [5]:
found = 0
total_prec = 0
for i in tqdm(range(len(cos_matrix))):
    sample = cos_matrix[i]
    results = list(sorted(enumerate(sample), key=lambda item: item[1], reverse=True))

    embd_set = set()
    total_list = []
    for r in results:
        if r[1] not in embd_set:
            embd_set.add(r[1])
            total_list.append(r)

    results = total_list[:10]

    count = 0
    precision = 0
    index = 0
    for m in results:
        if docs[m[0]] in pair[i][1]:
            count += 1
            precision += count/(index+1)
        index += 1

    if count == 0:
        precision = 0
    else:
        found += 1
        precision /= count

    total_prec += precision

100%|██████████| 2675/2675 [00:07<00:00, 363.20it/s]


In [6]:
MAP = total_prec / len(docs)
print(MAP)

0.17082289227166264


### 2.Recall

In [7]:
found = 0
total_prec = 0
for i in tqdm(range(len(cos_matrix))):
    sample = cos_matrix[i]
    results = list(sorted(enumerate(sample), key=lambda item: item[1], reverse=True))

    embd_set = set()
    total_list = []
    for r in results:
        if r[1] not in embd_set:
            embd_set.add(r[1])
            total_list.append(r)

    results = total_list[:10]

    count = 0
    for m in results:
        if docs[m[0]] in pair[i][1]:
            count += 1

    total_prec += (count / len(pair[i][1]))

100%|██████████| 2675/2675 [00:07<00:00, 357.88it/s]


In [8]:
RECALL = total_prec / len(docs)
print(RECALL)

0.30460655737704917


### 3.MRR

In [9]:
found = 0
total_prec = 0
for i in tqdm(range(len(cos_matrix))):
    sample = cos_matrix[i]
    results = list(sorted(enumerate(sample), key=lambda item: item[1], reverse=True))

    embd_set = set()
    total_list = []
    for r in results:
        if r[1] not in embd_set:
            embd_set.add(r[1])
            total_list.append(r)

    results = total_list[:10]

    index = 1
    for m in results:
        if docs[m[0]] in pair[i][1]:
            break
        index += 1

    total_prec += (1/index)

100%|██████████| 2675/2675 [00:07<00:00, 369.94it/s]


In [10]:
MRR = total_prec / len(docs)
print(MRR)

0.22263626901331723
